Assigmnent was divided into 2 parts. 
Part I: analysis of churning customers, answers were given in the format that was specifically asked by the task
Part II: analysis of tweets with prediction if an authour of the tweet will churn or not.

In [2]:
def write_txt(student_id, part_number, data):
    file = open("ID_{0}_Q_{1}.txt".format(student_id, part_number), 'w')
    file.write(data)
    file.close()

# Part I

In [22]:
#Import data

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import json

data = pd.read_json('churn_survey.json')
data.head()

,Churn,Contract,Dependents,DeviceProtection,Gender,InternetService,MonthlyCharges,MultipleLines,OnlineBackup,OnlineSecurity,PaperlessBilling,Partner,PaymentMethod,PhoneService,SeniorCitizen,StreamingMovies,StreamingTV,TechSupport,Tenure,TotalCharges
0,No,Month-to-month,No,No,Female,DSL,50.15,No,No,Yes,No,No,Electronic check,Yes,No,No,No,No,1.0,50.15
1,No,One year,No,Yes,Male,DSL,69.00,Yes,No,No,No,Yes,Credit card (automatic),Yes,No,Yes,No,Yes,38.0,2669.45
2,Yes,Month-to-month,Yes,No,Male,Fiber optic,74.35,Yes,No,No,Yes,Yes,Electronic check,Yes,No,No,No,No,11.0,834.20
3,No,Month-to-month,Yes,Yes,Male,DSL,45.00,No phone service,Yes,Yes,Yes,No,Mailed check,No,No,No,No,Yes,29.0,1242.45
4,Yes,Month-to-month,Yes,No,Female,Fiber optic,79.60,Yes,Yes,No,Yes,Yes,Mailed check,Yes,Yes,No,No,No,34.0,2718.30


In [34]:
#1.1 What is the most popular payment method?
data.groupby('PaymentMethod').describe()

#method_name = "Electronic check"

MonthlyCharges                               \
                                   count       mean        std    min   
PaymentMethod                                                           
Bank transfer (automatic)          760.0  66.651447  30.987177  18.40   
Credit card (automatic)            802.0  65.481920  30.867801  18.55   
Electronic check                  1159.0  77.094996  23.728916  19.05   
Mailed check                       799.0  44.109074  26.396697  18.70   

                                                            Tenure             \
                               25%    50%      75%     max   count       mean   
PaymentMethod                                                                   
Bank transfer (automatic)  35.3625  72.85  92.7375  118.75   760.0  42.836842   
Credit card (automatic)    34.8250  70.75  90.5625  117.50   802.0  42.683292   
Electronic check           63.2500  81.00  95.1000  118.65  1159.0  25.289905   
Mailed check               20.1250  35.00  64.9750  115.10   798.0  21.056391   

                            ...                TotalCharges               \
                            ...      75%   max        count         mean   
PaymentMethod               ...                                            
Bank transfer (automatic)   ...     66.0  72.0        758.0  2999.170515   
Credit card (automatic)     ...     66.0  72.0        801.0  2998.852809   
Electronic check            ...     43.0  72.0       1159.0  2121.026704   
Mailed check                ...     35.0  72.0        794.0  1004.880919   

                                                                             \
                                   std    min      25%       50%        75%   
PaymentMethod                                                                 
Bank transfer (automatic)  2334.287186  19.25  996.575  2402.950  4804.7250   
Credit card (automatic)    2419.750687  19.30  963.950  2250.650  4965.1000   
Electronic check           2176.203277  19.30  320.450  1258.350  3416.7250   
Mailed check               1383.506748  18.90  114.250   460.075  1266.6625   

                                    
                               max  
PaymentMethod                       
Bank transfer (automatic)  8684.80  
Credit card (automatic)    8670.10  
Electronic check           8564.75  
Mailed check               8075.35  

[4 rows x 24 columns]

In [5]:
#1.2 What is the mean amount spent per month for the churn and non-churn customers?

mean_amount_spent = data.groupby('Churn').mean()['MonthlyCharges'].round(4)

mean_amount_spent = mean_amount_spent.reset_index()
mean_amount_spent

,Churn,MonthlyCharges
0,No,61.1135
1,Yes,74.6525


In [6]:
#1.3 What is the standard deviation of amount spent per month for the churn and non-churn customers?

std_amount_spent = data.groupby('Churn').std()['MonthlyCharges'].round(4)

std_amount_spent = std_amount_spent.reset_index()
std_amount_spent

,Churn,MonthlyCharges
0,No,31.146
1,Yes,25.065


In [7]:
#1.4 What is the percentage of contract type for the churn and non-churn customers

churn = data.groupby('Churn')

percent_contract_type = churn['Contract'].value_counts(normalize = True).mul(100).round(2).unstack()
percent_contract_type

Contract,Month-to-month,One year,Two year
Churn,,,
No,43.87,25.01,31.12
Yes,88.53,8.36,3.11


In [8]:
#1.5 Which feature and value is most correlated with MonthlyCharges?

dummies_data = pd.get_dummies(data)

most_corr_feat = dummies_data.corr().stack().reset_index(name='Value').sort_values(by='Value',ascending=False)

most_corr_feat = most_corr_feat[most_corr_feat.Value != 1]
most_corr_feat = most_corr_feat[most_corr_feat.level_0 == 'MonthlyCharges']
most_corr_feat = most_corr_feat.rename(columns={"level_1": "Feature"}).head(1).loc[:,['Feature','Value']]

most_corr_feat

,Feature,Value
16,InternetService_Fiber optic,0.794367


In [9]:
#1.6 What is the count, mean, std, min, quartiles and max of time before a customer churns?

tenure_info = data.describe().round(4).loc[:,['Tenure']]

tenure_info

,Tenure
count,3520.0000
mean,32.0747
std,24.6002
min,0.0000
25%,8.0000
50%,28.5000
75%,55.0000
max,72.0000


In [10]:
#1.7 What is the proportion of purchase for each account addon for male and female customers?

addons = ["DeviceProtection", "OnlineBackup", "OnlineSecurity", "StreamingMovies", "StreamingTV", "TechSupport"]

dfs = []

for i in addons:
    addon1 = data[data[i] == "Yes"]
    n=len(addon1)
    a=addon1.groupby('Gender').count().div(n).round(4)
    a=a.loc[['Female', 'Male'],i]
    dfs.append(a)
    
result = pd.concat(dfs, axis=1)

result

,DeviceProtection,OnlineBackup,OnlineSecurity,StreamingMovies,StreamingTV,TechSupport
Gender,,,,,,
Female,0.5063,0.513,0.5228,0.5176,0.5054,0.5074
Male,0.4937,0.487,0.4772,0.4824,0.4946,0.4926


In [11]:
#1.8 Of the listed account addons, which addon/addons could be offered to churning customers for free or at a discounted rate in order to best retain them?

addons = ["DeviceProtection", "OnlineBackup", "OnlineSecurity", "StreamingMovies", "StreamingTV", "TechSupport"]
dfs_churn = []

for i in addons:
    addon1 = data[data[i] == "Yes"]
    n=len(data)
    b=addon1.groupby('Churn').count().div(n).round(4)
    b=b.loc[['Yes', 'No'],i]
    dfs_churn.append(b)
result_churn = pd.concat(dfs_churn, axis=1)
result_churn = result_churn.T

result_churn['Difference'] = result_churn['No'] - result_churn['Yes']

result_churn_sorted = result_churn.sort_values('Difference', ascending = False).reset_index().drop(['Yes','No','Difference'],axis=1).rename(columns={"index": "Addon"}).head(4)

result_churn_sorted

Churn,Addon
0,TechSupport
1,OnlineSecurity
2,DeviceProtection
3,OnlineBackup


# Part II

In [23]:
import re
import sqlite3

conn = sqlite3.connect("tweets.db")

tweet_df = pd.read_sql_query("SELECT * FROM tweets", conn)
churn_df = pd.read_sql("SELECT * FROM churn", conn)

tweet_df.set_index('tid', drop=True, inplace=True)
churn_df.set_index('tid', drop=True, inplace=True)

dd = churn_df.join(tweet_df)

training = dd[dd["set"] == "training"]
test = dd[dd["set"] == "hidden"]

In [24]:
#2.1 How many tweets in the training set contain at least one of the strings Verizon|AT&T|T-mobile.

regexp2_1 = 'Verizon|AT&T|T-mobile'

tweets_contain_company = training.loc[training.text.str.contains(regexp2_1, case=False, na=False), :]

number_tweets = str(len(tweets_contain_company))

number_tweets

'2492'

In [25]:
#2.2 Find all tweets in the training set containing the strings

regexp2_2 = 'Verizon.*AT&T.*switch|Verizon.*switch.*AT&T|AT&T.*Verizon.*switch|AT&T.*switch.*Verizon|switch.*Verizon.*AT&T|switch.*AT&T.*Verizon'

tweets_contain_switch = training.loc[training.text.str.contains(regexp2_2, case=False, na= False), :]

output_switch = tweets_contain_switch.drop(['date','uid','churn','set'], axis=1)

output_switch.head()

,text
tid,
3.145813e+17,Im about to switch to at&t if this keeps up; @...
4.210400e+17,T-Mobile Will Cover Switching Fees To Steal Mo...
4.212838e+17,Get out ya wack sprint at&t and verizon contra...
3.837465e+17,@imjust_jabriel I got that same shit with at&t...
3.390231e+17,@PhoneDog_Chase Trying to decide if I should s...


In [26]:
#2.3.1  Transform Features

from sklearn.feature_extraction.text import TfidfVectorizer
tdif_vectorizer = TfidfVectorizer(stop_words="english")

text = training['text'].dropna()
text = text.tolist()

X = tdif_vectorizer.fit_transform(text)

import pickle
filename = "ID_{0}_Q_2_3_1.pickle".format(480107600)
s = pickle.dump(tdif_vectorizer, open(filename, 'wb'))

In [27]:
#2.3.2 Dimension Reduction

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import numpy as np

tsvd = TruncatedSVD(n_components=5)

dimen_reduced = tsvd.fit(X).transform(X)

filename = "ID_{0}_Q_2_3_2.pickle".format(480107600)
s = pickle.dump(tsvd, open(filename, 'wb'))

In [28]:
#2.3.3 Tuning

training = training.iloc[0:2492,:]

X_train, X_test, y_train, y_test = train_test_split(dimen_reduced, training['churn'], test_size=0, random_state=42)

model = linear_model.LogisticRegression()
param_grid = { 
            "penalty": ['l1', 'l2'],
            "class_weight": [None, 'balanced'],
            "fit_intercept": [True, False],
            "C": np.logspace(-3, 3, 7), 
            "tol": [0.00001, 0.0001, 0.01, 1.0]
            }

grid_cv_obj = GridSearchCV(model, param_grid, return_train_score=True, cv=3)

grid_cv_obj.fit(X_train, y_train)

best_params = grid_cv_obj.best_params_

best_params

{'C': 10.0,
 'class_weight': None,
 'fit_intercept': True,
 'penalty': 'l1',
 'tol': 1.0}

In [29]:
#2.3.4 Output Model

clf_logreg = linear_model.LogisticRegression(random_state = 42).set_params(**best_params)
clf_logreg = clf_logreg.fit(X_train, y_train)

filename = "ID_{0}_Q_2_3_4.pickle".format(YOURSTUDENTID)
s = pickle.dump(clf_logreg, open(filename, 'wb'))

In [30]:
#2.3.5 Predicting Churn for the Hidden Customers 

tdif_vectorizer = TfidfVectorizer(stop_words="english")

text_test = test['text'].dropna()
text_test.tolist()

X_test = tdif_vectorizer.fit_transform(text_test)

tsvd_test = TruncatedSVD(n_components=5)

dimen_reduced_test = tsvd.fit(X_test).transform(X_test)
predictions = clf_logreg.predict(dimen_reduced_test)

test["Churn"] = predictions

answer_to_buss6002 = test.loc[:,['tid','Churn']]
answer_to_buss6002.head()

,tid,Churn
tid,,
3.259585e+17,NaN,0.0
3.245397e+17,NaN,0.0
3.272888e+17,NaN,0.0
3.472147e+17,NaN,0.0
3.958688e+17,NaN,0.0


In [31]:
#2.4 Prediction Competition based on test set

from sklearn.tree import DecisionTreeClassifier

clrTree = DecisionTreeClassifier()
clrTree = clrTree.fit(dimen_reduced, y_train)
outTree = clrTree.predict(dimen_reduced_test)
test["Churn"] = outTree
last_answer = test.loc[:,['Churn']]

last_answer.head()

,Churn
tid,
3.259585e+17,0.0
3.245397e+17,0.0
3.272888e+17,1.0
3.472147e+17,0.0
3.958688e+17,0.0
